In [ ]:
# USE O GOOGLE COLAB
# Instale as bibliotecas necessárias
!pip install pandas openpyxl requests tqdm

import pandas as pd
import requests
import re
import time
from tqdm import tqdm

# Suba o arquivo da planilha de CEPs
from google.colab import files
uploaded = files.upload()

# Nome do arquivo da planilha que você subiu
file_name = list(uploaded.keys())[0]

# Leia a planilha de entrada usando pandas, garantindo que os CEPs sejam lidos como strings
df = pd.read_excel(file_name, dtype={'CEP': str})

# Remova quaisquer caracteres não numéricos dos CEPs, preservando zeros à esquerda
df['CEP'] = df['CEP'].apply(lambda x: re.sub(r'\D', '', str(x)).zfill(8))

# Função para obter latitude e longitude usando a API do Nominatim
def get_lat_long_nominatim(cep, retries=3, pause=1):
    headers = {
        "User-Agent": "Colab-geocoder-1.0"
    }
    for attempt in range(retries):
        try:
            url = f"https://nominatim.openstreetmap.org/search?postalcode={cep}&country=Brazil&format=json&limit=1"
            response = requests.get(url, headers=headers, timeout=10)
            data = response.json()
            if data:
                # Retorna latitude e longitude do primeiro resultado
                return float(data[0]['lat']), float(data[0]['lon'])
            else:
                print(f"Erro ao processar o CEP {cep}: sem resultados")
                return None, None
        except requests.exceptions.RequestException as e:
            print(f"Erro ao processar o CEP {cep} na tentativa {attempt + 1}: {e}")
            time.sleep(pause)
    return None, None

# Crie duas novas colunas para armazenar as coordenadas, com barra de progresso
latitudes = []
longitudes = []
for cep in tqdm(df['CEP'], desc="Processando CEPs"):
    lat, lon = get_lat_long_nominatim(cep)
    latitudes.append(lat)
    longitudes.append(lon)

# Adiciona as colunas de Latitude e Longitude ao DataFrame
df['Latitude'] = latitudes
df['Longitude'] = longitudes

# Substitui vírgulas por pontos para garantir o formato numérico correto
df['Latitude'] = df['Latitude'].apply(lambda x: str(x).replace(',', '.') if x is not None else x)
df['Longitude'] = df['Longitude'].apply(lambda x: str(x).replace(',', '.') if x is not None else x)

# Salve os resultados em uma nova planilha
output_file = 'ceps_com_coordenadas_nominatim.xlsx'
df.to_excel(output_file, index=False)

# Baixe o arquivo resultante
files.download(output_file)
